In [1]:
import objgraph 
import pygame
from Projectile import Projectile, Bomb
from Enemy import Enemy, EnemyFactory
from Player import Player
from  Weapon import WeaponFactory
from Weapon import Weapon
import gc
import tracemalloc

pygame 2.5.2 (SDL 2.28.3, Python 3.11.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class NoFlyWeightEnemy():
    projectiles = pygame.sprite.Group()
    def __init__(self, name: str, image: pygame.Surface, weapon_name: str, speed: float, 
                 default_health: int, pos: tuple[float, float]):
        super().__init__()
        self.image = pygame.transform.scale(pygame.image.load("./Images/smallEnemy.png"), (50,50))
        self.rect = self.image.get_rect()
        self.speed = speed
        self.weapon = WeaponFactory.get(weapon_name)

        self.name = name
        self.health = default_health
        self.movement_vector = [0, 0]
        self.pos = list(pos)
        self.last_shot_time = pygame.time.get_ticks()
        
    def move(self, enemies: pygame.sprite.Group, player_pos: tuple[float, float], tDelta: float):
        self.movement_vector = (player_pos[0] - self.pos[0],
                               player_pos[1] - self.pos[1])
        self.movement_vector = Weapon.normalize_vector(self.movement_vector)
        self.pos[0] += self.movement_vector[0] * self.speed * tDelta
        self.pos[1] += self.movement_vector[1] * self.speed * tDelta
        
        # Collision test with other enemies
        self.movement_vector = [0, 0]
        for sprite in enemies:
            if sprite is self:
                continue
            if pygame.sprite.collide_circle(self, sprite):
                self.movement_vector[0] += self.pos[0] - sprite.pos[0]
                self.movement_vector[1] += self.pos[1] - sprite.pos[1]

        self.movement_vector = Weapon.normalize_vector(self.movement_vector)
        self.pos[0] += self.movement_vector[0] * 0.5  # The constant is how far the sprite will be
        self.pos[1] += self.movement_vector[1] * 0.5  # dragged from the sprite it collided with
        
        self.rect.topleft = self.pos

    def attack(self, target_pos):
        self.weapon.attack(self, target_pos, self.last_shot_time)
        self.last_shot_time = pygame.time.get_ticks()

    def render(self, surface):
        surface.blit(self.image, self.pos)
    

In [3]:
# Display memory growth before creating instances
print("Memory usage before creating instances:")
objgraph.show_growth()

# create more instances of each enemy type
# edit to 1000 for good example
for i in range(10):
    enemy = Enemy(name = "small", pos=(0,0))

# Display memory growth after creating instances
print("Memory usage after creating instances:")
objgraph.show_growth()


Memory usage before creating instances:
function                      21857    +21857
tuple                         12298    +12298
dict                          10806    +10806
list                           5901     +5901
ReferenceType                  5875     +5875
builtin_function_or_method     3725     +3725
wrapper_descriptor             3255     +3255
cell                           3024     +3024
getset_descriptor              2970     +2970
type                           2807     +2807
Memory usage after creating instances:
list            5902        +1
set             1378        +1
Enemy              1        +1
Interactive        1        +1


In [4]:
#Display memory growth before creating instances
print("Memory usage before creating instances:")
objgraph.show_growth()

# edit to 1000 for a good example
for i in range(10):
    no_flyweight_enemy = NoFlyWeightEnemy(name = "small", image=pygame.image.load("./Images/smallEnemy.png"), weapon_name="small", speed=1, default_health=1, pos=(0,0))

# Display memory growth after creating instances
print("Memory usage after creating instances:")
objgraph.show_growth()

Memory usage before creating instances:
dict                          10814        +8
tuple                         12303        +5
list                           5906        +4
Constant                         10        +4
keyword                           6        +4
builtin_function_or_method     3727        +2
Attribute                         4        +2
ReferenceType                  5876        +1
method                         2447        +1
partial                          41        +1
Memory usage after creating instances:
cell                 3027        +3
list                 5907        +1
dict                10815        +1
function            21858        +1
NoFlyWeightEnemy        1        +1


In [5]:
# Display memory growth before creating instances
print("Memory usage before creating instances:")
objgraph.show_growth()

# edit to 1000 for a good example
for i in range(10):
    enemyFlyweight = EnemyFactory.get("small")

# Display memory growth after creating instances
print("Memory usage after creating instances:")
objgraph.show_growth()

Memory usage before creating instances:
ReferenceType     5877        +1
tuple            12304        +1
Memory usage after creating instances:
method     2448        +1


In [6]:
# Start tracing
tracemalloc.start()

# Create instances of the Enemy class and measure memory growth
enemy_memory_growth = []

for i in range(1, 1001):  # Change the range based on your needs
    enemyFlyweight = EnemyFactory.get("small")

    # Measure memory usage after creating each instance
    current_memory = tracemalloc.get_traced_memory()[1] / 1024.0  # Convert to kilobytes
    enemy_memory_growth.append((i, current_memory))

# Stop tracing
tracemalloc.stop()

# Display memory growth over time
for i, memory in enemy_memory_growth:
    print(f"Enemy {i}: Memory = {memory:.2f} KB")

Enemy 1: Memory = 15.72 KB
Enemy 2: Memory = 15.72 KB
Enemy 3: Memory = 15.72 KB
Enemy 4: Memory = 15.72 KB
Enemy 5: Memory = 15.72 KB
Enemy 6: Memory = 15.72 KB
Enemy 7: Memory = 15.72 KB
Enemy 8: Memory = 15.72 KB
Enemy 9: Memory = 15.72 KB
Enemy 10: Memory = 15.72 KB
Enemy 11: Memory = 15.72 KB
Enemy 12: Memory = 15.72 KB
Enemy 13: Memory = 15.72 KB
Enemy 14: Memory = 15.72 KB
Enemy 15: Memory = 15.72 KB
Enemy 16: Memory = 15.72 KB
Enemy 17: Memory = 15.72 KB
Enemy 18: Memory = 15.72 KB
Enemy 19: Memory = 15.72 KB
Enemy 20: Memory = 15.72 KB
Enemy 21: Memory = 15.72 KB
Enemy 22: Memory = 15.72 KB
Enemy 23: Memory = 15.72 KB
Enemy 24: Memory = 15.72 KB
Enemy 25: Memory = 15.72 KB
Enemy 26: Memory = 15.72 KB
Enemy 27: Memory = 15.72 KB
Enemy 28: Memory = 15.72 KB
Enemy 29: Memory = 15.72 KB
Enemy 30: Memory = 15.72 KB
Enemy 31: Memory = 15.72 KB
Enemy 32: Memory = 15.72 KB
Enemy 33: Memory = 15.72 KB
Enemy 34: Memory = 15.72 KB
Enemy 35: Memory = 15.72 KB
Enemy 36: Memory = 15.72 KB
E

In [7]:
# Start tracing
tracemalloc.start()

# Create instances of the Enemy class and measure memory growth
enemy_memory_growth = []

for i in range(1, 1001):  # Change the range based on your needs
    enemy = Enemy(name="small", pos=(0, 0))

    # Measure memory usage after creating each instance
    current_memory = tracemalloc.get_traced_memory()[1] / 1024.0  # Convert to kilobytes
    enemy_memory_growth.append((i, current_memory))

# Stop tracing
tracemalloc.stop()

# Display memory growth over time
for i, memory in enemy_memory_growth:
    print(f"Enemy {i}: Memory = {memory:.2f} KB")

Enemy 1: Memory = 15.86 KB
Enemy 2: Memory = 15.86 KB
Enemy 3: Memory = 15.86 KB
Enemy 4: Memory = 15.86 KB
Enemy 5: Memory = 15.86 KB
Enemy 6: Memory = 15.86 KB
Enemy 7: Memory = 15.86 KB
Enemy 8: Memory = 15.86 KB
Enemy 9: Memory = 15.86 KB
Enemy 10: Memory = 15.86 KB
Enemy 11: Memory = 15.86 KB
Enemy 12: Memory = 15.86 KB
Enemy 13: Memory = 15.86 KB
Enemy 14: Memory = 15.86 KB
Enemy 15: Memory = 15.86 KB
Enemy 16: Memory = 15.86 KB
Enemy 17: Memory = 15.86 KB
Enemy 18: Memory = 15.86 KB
Enemy 19: Memory = 15.86 KB
Enemy 20: Memory = 15.86 KB
Enemy 21: Memory = 15.86 KB
Enemy 22: Memory = 15.86 KB
Enemy 23: Memory = 15.86 KB
Enemy 24: Memory = 15.86 KB
Enemy 25: Memory = 15.86 KB
Enemy 26: Memory = 15.86 KB
Enemy 27: Memory = 15.86 KB
Enemy 28: Memory = 15.86 KB
Enemy 29: Memory = 15.86 KB
Enemy 30: Memory = 15.86 KB
Enemy 31: Memory = 15.86 KB
Enemy 32: Memory = 15.86 KB
Enemy 33: Memory = 15.86 KB
Enemy 34: Memory = 15.86 KB
Enemy 35: Memory = 15.86 KB
Enemy 36: Memory = 15.86 KB
E

In [8]:
 # Start tracing
tracemalloc.start()

# Create instances of the Enemy class and measure memory growth
enemy_memory_growth = []

for i in range(1, 1001):  # Change the range based on your needs
    no_flyweight_enemy = NoFlyWeightEnemy(name = "small", image=pygame.image.load("./Images/smallEnemy.png"), weapon_name="small", speed=1, default_health=1, pos=(0,0))

    # Measure memory usage after creating each instance
    current_memory = tracemalloc.get_traced_memory()[1] / 1024.0  # Convert to kilobytes
    enemy_memory_growth.append((i, current_memory))

# Stop tracing
tracemalloc.stop()

# Display memory growth over time
for i, memory in enemy_memory_growth:
    print(f"Enemy {i}: Memory = {memory:.2f} KB")
    

Enemy 1: Memory = 17.57 KB
Enemy 2: Memory = 17.57 KB
Enemy 3: Memory = 17.57 KB
Enemy 4: Memory = 17.57 KB
Enemy 5: Memory = 17.57 KB
Enemy 6: Memory = 17.57 KB
Enemy 7: Memory = 17.57 KB
Enemy 8: Memory = 17.57 KB
Enemy 9: Memory = 17.57 KB
Enemy 10: Memory = 17.57 KB
Enemy 11: Memory = 17.57 KB
Enemy 12: Memory = 17.57 KB
Enemy 13: Memory = 17.57 KB
Enemy 14: Memory = 17.57 KB
Enemy 15: Memory = 17.57 KB
Enemy 16: Memory = 17.57 KB
Enemy 17: Memory = 17.57 KB
Enemy 18: Memory = 17.57 KB
Enemy 19: Memory = 17.57 KB
Enemy 20: Memory = 17.57 KB
Enemy 21: Memory = 17.57 KB
Enemy 22: Memory = 17.57 KB
Enemy 23: Memory = 17.57 KB
Enemy 24: Memory = 17.57 KB
Enemy 25: Memory = 17.57 KB
Enemy 26: Memory = 17.57 KB
Enemy 27: Memory = 17.57 KB
Enemy 28: Memory = 17.57 KB
Enemy 29: Memory = 17.57 KB
Enemy 30: Memory = 17.57 KB
Enemy 31: Memory = 17.57 KB
Enemy 32: Memory = 17.57 KB
Enemy 33: Memory = 17.57 KB
Enemy 34: Memory = 17.57 KB
Enemy 35: Memory = 17.57 KB
Enemy 36: Memory = 17.57 KB
E